# Mersenne Cryptosystem

We impliment the Mersenne-based cryptosystem proposed by Aggarwal, Joux,Prakash, and Santha ([CRYPTO 2017](https://eprint.iacr.org/2017/481.pdf)).

## 1-bit Message Mersenne PKE

In [1]:
import random
import numpy as np

### Prime Choice Algorithm

In [2]:
def ith_Mersenne_prime_choice(i):
	MPexplst = [2,3,5,7,13,17,19,31,61,89,107,127,521,607,1279,2203,2281,3217,4253,4423,9689,9941,11213,19937,21701,23209,44497,86243,110503,132049]
	bitlength = MPexplst[i]
	p = 2**MPexplst[i] - 1
	return(bitlength, p)

### Sampling Algorithm for n-bit Strings with Hamming Weight h

In [3]:
def choice_nbitstr_hweight(n, h):
	b1plst = random.sample(range(n), h)
	nhstring = [0]*n
	for i in range(len(b1plst)):
		place = b1plst[i]
		nhstring[place] = 1
	return(nhstring)

### Conversion between a Bit String and an Integer

In [4]:
def str_to_int(string, n):
	integer = 0
	revstr = string[::-1]
	for i in range(len(revstr)):
		integer = integer + revstr[i] * 2 ** (i)
	integer = integer % (2 ** n - 1)
	return(integer)

def int_to_str(inte, n):
	midinte = inte % (2 ** n - 1)
	string = [0]*n
	for i in reversed(range(n)):
		if ((midinte // 2 ** i) == 1):
			string[i] = 1
			midinte = midinte - (2 ** i)
		else:
			string[i] = 0
	revstr = string[::-1]
	return(revstr)

### Hamming Weight Mesureing Algorithm

In [5]:
def Hamweight(lst):
	ctr = 0
	for i in range(len(lst)):
		if lst[i] != 0:
			ctr = ctr + 1
	return(ctr)

### Extended Euclid Algotithm and Inverse Element Computation algorithm

In [6]:
def exgcd(a, b):
    x0, y0, x1, y1 = 1, 0, 0, 1
    while b != 0:
        q, a, b = a // b, b, a % b
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return a, x0, y0

def modinv(a, p):
    g, x, y = exgcd(a, p)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return (x % p)

### 1-bit PKE Cryptosystem Algorithm

In [7]:
def one_bit_pke_keygen(n, h):
	strF = choice_nbitstr_hweight(n, h)
	strG = choice_nbitstr_hweight(n, h)
	strH = int_to_str((str_to_int(strF, n) * modinv(str_to_int(strG, n), p) % p), n)
	return(strG,strH)

def one_bit_pke_enc(n, h, p, strH, bit):
	if (bit == 0 or bit ==1):
		strA = choice_nbitstr_hweight(n, h)
		strB = choice_nbitstr_hweight(n, h)
		strC = int_to_str((-1) ** bit * (str_to_int(strA, n) * str_to_int(strH, n) + str_to_int(strB, n) % p), n)
		return(strC)
	else:
		return('invalid')

def one_bit_pke_dec(n, h, p, strG ,strC):
	strD = int_to_str(((str_to_int(strC, n) * str_to_int(strG, n)) % p), n)
	dbit = Hamweight(strD)
	if dbit <= 2 * h ** 2:
		return(0)
	elif dbit >= (n - (2 * h ** 2)):
		return(1)
	else:
		return('invalid')
	return(dbit)

### Parameters and Message Choice

In [8]:
i = 14 # i represents the i-th mersenne prime. (i = 11, n = 127) (i=12, n=521) (i=14, n=1278)

(n, p) =(ith_Mersenne_prime_choice(i))

print(f'n = {n}')
print(f'p = {p}')
print(f'We must choose h such that 4h^2 < {n} <= 16h^2')
lowh = np.sqrt(n /16)
highh = np.sqrt(n /4)
print(f'That is, we should choose h such that {lowh} <=  h < {highh}')

n = 1279
p = 10407932194664399081925240327364085538615262247266704805319112350403608059673360298012239441732324184842421613954281007791383566248323464908139906605677320762924129509389220345773183349661583550472959420547689811211693677147548478866962501384438260291732348885311160828538416585028255604666224831890918801847068222203140521026698435488732958028878050869736186900714720710555703168729087
We must choose h such that 4h^2 < 1279 <= 16h^2
That is, we should choose h such that 8.940777371123833 <=  h < 17.881554742247666


In [9]:
# Choose parametr h
h = 10

# Choose message bit
mbit = 0

### 1-bit PKE  Cryptosystem Running

In [10]:
(skG, pkH) = one_bit_pke_keygen(n, h)
cipher = one_bit_pke_enc(n, h, p, pkH, mbit)
dbit = one_bit_pke_dec(n, h, p, skG ,cipher)

print(f'public key (pkH = {pkH})')
print(f'public key (skH = {skG})')
print(f'message bit b = {mbit}')
print(f'cipertext (C = {cipher})')
print(f'decryption result d = {dbit}')

public key (pkH = [1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 